In [1]:
import pandas as pd
import numpy as np
import requests
import json

In [2]:
def authToken(url_key,username,password):
    r1 = requests.post(url=url_key,data=json.dumps({"username":username,"password":password}),headers={"Content-Type":"application/json"})
    if r1.status_code == 400:
        return False
    return r1.json()
base_cicero_url = "https://cicerosupply.com"
ext = "/rest/V1/integration/admin/token"
global cicero_token
cicero_token = authToken(base_cicero_url+ext,"Shruti","@bQSPTzR7bpJ")
cicero_token

'iaka1vgpfxka6d1bk9q6e3jj3kk8a6y7'

In [3]:
url = "https://cicerosupply.com/rest/V1/products/national cycle"
headers = {"Authorization":"Bearer {}".format(cicero_token),\
        "Content-Type": "application/json"}
response = requests.get(url=url,headers=headers)

In [4]:
indi_skus = [i["linked_product_sku"] for i in response.json()["product_links"]]
len(indi_skus)

280

In [5]:
url_all_prods = "https://cicerosupply.com/rest/V1/products?searchCriteria[filterGroups][0][filters][0][field]=sku&searchCriteria[filterGroups][0][filters][0][condition_type]=in&searchCriteria[filterGroups][0][filters][0][value]="+",".join(indi_skus)
all_prods = requests.get(url=url_all_prods,headers=headers)

In [19]:
prices = pd.DataFrame()
for i in all_prods.json()["items"]:
    sku = i["sku"].strip()
    price = i["price"]
    prices = pd.concat([prices,pd.DataFrame([sku,price]).T])
prices.reset_index(inplace=True,drop=True)
prices.columns = ["SKU","Price"]
prices.head()

,SKU,Price
0,10-122071-000,49.27
1,15-150110-000,12.66
2,15F538,23.49
3,1CH72,3
4,1VAC4,13.02


In [20]:
# prices.to_csv("Output/indi_nc_prices.csv",index=None)

In [6]:
custom_attribute_list = []
for i in all_prods.json()["items"]:
    for j in i["custom_attributes"]:
        if j["attribute_code"] not in custom_attribute_list:
            custom_attribute_list.append(j["attribute_code"])

In [7]:
custom_attribute_list

['manufacturer',
 'url_key',
 'gift_message_available',
 'unit',
 'meta_title',
 'options_container',
 'meta_keyword',
 'increment_of',
 'required_options',
 'has_options',
 'tax_class_id',
 'category_ids',
 'mst_search_weight',
 'description',
 'msrp_display_actual_price_type',
 'short_description',
 'meta_description']

In [43]:
for i in all_prods.json()["items"]:
    for j in i["custom_attributes"]:
        if j["attribute_code"]=="manufacturer":
            if j["value"]!="329":
                print(j["value"])
        if j["attribute_code"] in ['required_options','has_options']:
            if j["value"]!="0":
                print(j["attribute_code"],"=",j["value"])

In [9]:
all_prods.json()["items"][0]

{'id': 94838,
 'sku': '10-122071-000',
 'name': '18 X 23-1/2 Self-Seal Bubble Pouches (100)',
 'attribute_set_id': 39,
 'price': 49.27,
 'status': 1,
 'visibility': 1,
 'type_id': 'simple',
 'created_at': '2018-01-12 15:51:58',
 'updated_at': '2018-12-27 10:59:45',
 'extension_attributes': {'website_ids': [1],
  'category_links': [{'position': 1, 'category_id': '1446'},
   {'position': 1, 'category_id': '1448'}]},
 'product_links': [],
 'options': [],
 'media_gallery_entries': [],
 'tier_prices': [],
 'custom_attributes': [{'attribute_code': 'manufacturer', 'value': '329'},
  {'attribute_code': 'url_key',
   'value': '18-x-23-1-2-self-seal-bubble-pouches-100'},
  {'attribute_code': 'gift_message_available', 'value': '2'},
  {'attribute_code': 'unit', 'value': 'Case'},
  {'attribute_code': 'meta_title',
   'value': 'Tridim 24 X 12 X 2 Merv 7 Filter'},
  {'attribute_code': 'options_container', 'value': 'container2'},
  {'attribute_code': 'meta_keyword',
   'value': 'Tridim 24 X 12 X 2 Me

In [17]:
def getBrandsList():
    url = "https://cicerosupply.com/rest/V1/products/attributes/manufacturer/options"
    headers = {'Authorization': 'Bearer '+cicero_token}
    response = requests.request("GET", url, headers=headers)
    brand_dict = dict()
    for i in response.json():
        if i["label"]==" ":
            continue
        brand_dict[i["value"]] = i["label"]
    
    return brand_dict
brands = getBrandsList()
brands

{'4': '3M',
 '5': 'Loctite',
 '6': 'Zep Inc.',
 '7': 'Apex Tool Group',
 '8': 'JPW Industries',
 '10': 'Magnolia Brush',
 '11': 'Advance Brush',
 '12': 'Anchor Products',
 '13': 'Anderson Brush',
 '20': 'Red Devil',
 '14': 'Ampco Safety Tools',
 '15': 'Bee Line',
 '16': 'Norton',
 '17': 'WEILER',
 '18': 'Carborundum',
 '19': 'DEWALT',
 '21': 'Metabo',
 '22': 'STANLEY',
 '23': 'Merit Abrasives',
 '24': 'Spiralcool',
 '25': 'Ridge Tool Company',
 '26': 'Eagle Brush',
 '27': 'CGW Abrasives',
 '28': 'GMT',
 '29': 'Porter Cable',
 '30': 'DYNABRADE',
 '31': 'Franklin',
 '32': 'GREENLEE',
 '33': 'Bosch Tool Corporation',
 '34': 'Intertape Polymer Group',
 '35': 'Amrep Inc.',
 '36': 'Berry',
 '37': 'KRYLON',
 '38': 'Devcon',
 '39': 'J-B Weld',
 '40': 'Rectorseal',
 '41': 'Technetics Group',
 '42': 'DAP',
 '43': 'Tesa Tapes',
 '44': 'Shurtape',
 '45': 'Radiator Specialty',
 '46': 'Pferd',
 '47': 'Aquasol Corporation',
 '48': 'Permatex',
 '49': 'Aervoe Industries',
 '50': 'Sika Corporation',
 '5

In [36]:
#['sku','manufacturer_name','url_key','unit_of_measure','increment_of','description','short_description']
products = pd.DataFrame()
for i in all_prods.json()["items"]:
    sku = i["sku"]
    manufacturer= ""
    url_key= ""
    unit= ""
    inc_of= ""
    desc= ""
    sh_desc = ""
    name = i["name"]
    for j in i["custom_attributes"]:
        if j["attribute_code"] == "manufacturer":
            manufacturer = brands[j["value"]]
        elif j["attribute_code"] == "url_key":
            url_key = j["value"]
        elif j["attribute_code"] == "unit":
            unit = j["value"]
        elif j["attribute_code"] == "increment_of":
            inc_of = j["value"]
        elif j["attribute_code"] == "description":
            desc = j["value"]
        elif j["attribute_code"] == "short_description":
            sh_desc = j["value"]
    products = pd.concat([products, pd.DataFrame([sku,name,manufacturer,url_key,unit,inc_of,desc,sh_desc]).T])
len(products)


280

In [37]:
products.columns = ["sku","Name","Manufacturer Name","URL Key","Unit of Measure","Increment of","Description","Short Description"]
products["Brand"] = products["Manufacturer Name"]
products.head()

,sku,Name,Manufacturer Name,URL Key,Unit of Measure,Increment of,Description,Short Description,Brand
0,10-122071-000,18 X 23-1/2 Self-Seal Bubble Pouches (100),Delta Consolidated,18-x-23-1-2-self-seal-bubble-pouches-100,Case,1,,,Delta Consolidated
0,15-150110-000,Grease 41150 400G,Delta Consolidated,grease-41150-400g,Each,12,Grease 41150 400g (44N721) (order in increment...,,Delta Consolidated
0,15F538,Urinal Screens (12)\r\n,Delta Consolidated,urinal-screens-12,Box,1,\r\nUrinal Screens (12),,Delta Consolidated
0,1CH72,Trigger Sprayer,Delta Consolidated,trigger-sprayer,Each,1,Trigger Sprayer,,Delta Consolidated
0,1VAC4,Angle Brooms,Delta Consolidated,angle-brooms,Each,1,Angle Brooms,,Delta Consolidated


In [38]:
products.to_excel("Output/individualPOProds.xlsx",index=None)

In [15]:
# Use GenNCProductsPim.php sript to push these products to pim.

479